In [1]:
# Nama : MUHAMMAD NURUL ARIFIN
# Domisili : Gunungkidul, Yogyakarta

import tensorflow as tf
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing import image
import numpy as np
from google.colab import files
from sklearn.model_selection import train_test_split
import os
!pip install ipywidgets
import ipywidgets as widgets
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import time

dataset_validation = '/content/drive/MyDrive/Local Project/Dicoding/Dicoding-Machine learning untuk pemula/rockpaperscissors/val/'
dataset_train = '/content/drive/MyDrive/Local Project/Dicoding/Dicoding-Machine learning untuk pemula/rockpaperscissors/train/'


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.9 MB/s eta 0:00:00


In [2]:
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

image_list = []
label_list = []
class_labels = ['paper', 'rock', 'scissors']

for class_label in class_labels:
    class_dir = os.path.join(dataset_train,dataset_validation, class_label)
    for filename in os.listdir(class_dir):
        img = image.load_img(os.path.join(class_dir, filename), target_size=(150, 150))
        img = image.img_to_array(img)
        img = img / 255.0
        image_list.append(img)
        label_list.append(class_label)

In [3]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(label_list)

num_classes = len(label_encoder.classes_)
y_train_one_hot = to_categorical(y_train_encoded, num_classes=num_classes)


X_train, X_val, y_train, y_val = train_test_split(image_list, y_train_one_hot, test_size=0.2, random_state=42)


X_train = np.array(X_train)
X_val = np.array(X_val)
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(label_list)

num_classes = len(label_encoder.classes_)
y_train_one_hot = to_categorical(y_train_encoded, num_classes=num_classes)


X_train, X_val, y_train, y_val = train_test_split(image_list, y_train_one_hot, test_size=0.2, random_state=42)

X_train = np.array(X_train)
X_val = np.array(X_val)

In [4]:

model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(512, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])


model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_accuracy', mode='max')


In [5]:

history = model.fit(
    X_train,
    y_train,
    epochs=100,
    validation_data=(X_val, y_val),
    verbose=1,
    callbacks=[early_stopping, model_checkpoint]
)

Epoch 1/100
4/4 [==============================] - ETA: 0s - loss: 1.3979 - accuracy: 0.3083

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


4/4 [==============================] - 18s 868ms/step - loss: 1.3979 - accuracy: 0.3083 - val_loss: 1.0929 - val_accuracy: 0.3667
Epoch 2/100
4/4 [==============================] - 1s 218ms/step - loss: 1.0996 - accuracy: 0.3333 - val_loss: 1.0846 - val_accuracy: 0.4667
Epoch 3/100
4/4 [==============================] - 1s 236ms/step - loss: 1.0725 - accuracy: 0.4333 - val_loss: 1.0104 - val_accuracy: 0.6000
Epoch 4/100
4/4 [==============================] - 1s 271ms/step - loss: 0.8944 - accuracy: 0.6083 - val_loss: 0.8547 - val_accuracy: 0.6667
Epoch 5/100
4/4 [==============================] - 1s 278ms/step - loss: 0.7911 - accuracy: 0.7583 - val_loss: 0.6916 - val_accuracy: 0.7333
Epoch 6/100
4/4 [==============================] - 4s 1s/step - loss: 0.5842 - accuracy: 0.7667 - val_loss: 0.6639 - val_accuracy: 0.7667
Epoch 7/100
4/4 [==============================] - 1s 234ms/step - loss: 0.5421 - accuracy: 0.7833 - val_loss: 0.4978 - val_accuracy: 0.8333
Epoch 8/100
4/4 [==========

In [6]:

best_model = tf.keras.models.load_model('best_model.h5')

upload_button = widgets.FileUpload()
display(upload_button)

uploaded_image_dir = '/content/drive/MyDrive/Local Project/Dicoding/Dicoding-Machine learning untuk pemula/rockpaperscissors/img/'
def on_upload_button_click(change):
    uploaded_files = upload_button.value
    if uploaded_files:

        uploaded_file = list(uploaded_files.values())[0]


        timestamp = str(int(time.time()))
        uploaded_image_path = os.path.join(uploaded_image_dir, f'{timestamp}.png')

        with open(uploaded_image_path, 'wb') as f:
            f.write(uploaded_file['content'])


        img = image.load_img(uploaded_image_path, target_size=(150, 150))
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = img / 255.0


        predictions = best_model.predict(img)


        predicted_class = np.argmax(predictions)


        class_labels = ['paper', 'rock', 'scissors']
        predicted_label = class_labels[predicted_class]

        print(f"Predicted class: {predicted_label}")


upload_button.observe(on_upload_button_click, names='_counter')

FileUpload(value={}, description='Upload')

1/1 [==============================] - 0s 330ms/step
Predicted class: rock
1/1 [==============================] - 0s 30ms/step
Predicted class: rock
1/1 [==============================] - 0s 19ms/step
Predicted class: paper
1/1 [==============================] - 0s 20ms/step
Predicted class: scissors
